## Populate Inventory Data
The purpose of this workbook is to load-up all the flow information I hand-curated, and assign it to the flow entities.  Then I will use the metadata to assign (random) mass and economic value properties to the flows.

Later I will use the metainfo to terminate fragments using the flows.

In [1]:
from antelope_catalog import LcCatalog

In [2]:
cat = LcCatalog('/data/GitHub/2019/EPA_PSM_antelope/catalog/')

Loading JSON data from /data/GitHub/2019/EPA_PSM_antelope/catalog/reference-quantities.json:
local.qdb: /data/GitHub/2019/EPA_PSM_antelope/catalog/reference-quantities.json
local.qdb: /data/GitHub/lca-tools/lcatools/qdb/data/elcd_reference_quantities.json
6 new flow entities added (6 total)
25 new quantity entities added (25 total)


In [12]:
fg = cat.create_foreground('psm-hackathon')

In [13]:
ar = cat.get_archive(fg.origin)

foreground.psm-hackathon: /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon
Loading /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon
foreground.psm-hackathon: /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon/entities.json
local.usepa.useeio.1.1: /data/LCI/USEEIO/USEEIOv1.1-JSONLD-foropenlca.zip
Found Extension: zip
Applying stored configuration
Applying context hint local.usepa.useeio.1.1:water => to water
Applying context hint local.usepa.useeio.1.1:air => to air
Applying configuration to OpenLcaJsonLdArchive with 0 entities at /data/LCI/USEEIO/USEEIOv1.1-JSONLD-foropenlca.zip
Missing canonical quantity-- adding to LciaDb
registering local.usepa.useeio.1.1/b0682037-e878-4be4-a63a-a7a81053a691
Retrieving canonical [local.usepa.useeio.1.1] Producer Price US 2013 [USD]
1019 new flow entities added (1019 total)
0 new fragment entities added (0 total)
3 new quantity entities added (3 total)
1402 new fragment entities added (1402 total)


In [6]:
f = next(ar.entities_by_type('flow'))

In [7]:
from lcatools.entities.xlsx_editor import XlsxArchiveUpdater

In [8]:
f.show()

Flow Entity (ref 3M1169C3D6A)
origin: foreground.psm-hackathon
reference: [local.qdb] Number of items [Item(s)]
       Name: BOLT
    Comment: 
      Level: 1.0
  CasNumber: 
Compartment: []
         PT: 5


In [9]:
with XlsxArchiveUpdater(ar, 'PSM-hackathon-leaf-flows_annotated.xlsx') as x:
    x.apply()

In [17]:
f.show()

NameError: name 'f' is not defined

In [11]:
ar.save()

In [18]:
import sys
sys.path.append('/data/GitHub/2019/EPA_PSM_antelope/')

In [20]:
mass = cat.lcia_engine.get_canonical('mass')
USD = next(cat.query('local.usepa.useeio').quantities(Name='price'))

In [19]:
from antelope_epa.inventory_data import MockCfGenerator

In [22]:
mock = MockCfGenerator(mass, USD, 'PSM-hackathon-leaf-flows_annotated.xlsx')

In [24]:
_fgen = fg.flows(Material='.')

In [26]:
f = next(_fgen)
mock.characterize(f, demo=True)

In [29]:
USD.cf(f)

0.23534490398053493

In [30]:
for f in fg.flows(Material='.'):
    mock.characterize(f, demo=False)

In [31]:
mock.save()

Wrote CFs to cfs.json


In [33]:
ar.save()

In [35]:
qs = [q for q in ar.entities_by_type('quantity')]

In [37]:
qs[0].serialize()

{'origin': 'local.qdb',
 'externalId': 'flowproperties/01846770-4cfe-4a25-8ad9-919d8d378345',
 'entityType': 'quantity',
 'Name': 'Number of items',
 'UnitConversion': {'Dozen(s)': 0.08333333333333333, 'Item(s)': 1.0},
 'uuid': '01846770-4cfe-4a25-8ad9-919d8d378345',
 'referenceUnit': 'Item(s)'}

In [21]:
help(fg.characterize)

Help on method characterize in module lcatools.interfaces.iquantity:

characterize(flowable, ref_quantity, query_quantity, value, context=None, location='GLO', **kwargs) method of antelope_catalog.catalog_query.CatalogQuery instance
    Add Characterization data for a flowable, reporting the amount of a query quantity that is equal to a unit
    amount of a reference quantity, for a given context and location
    :param flowable: string or flow external ref
    :param ref_quantity: string or external ref
    :param query_quantity: string or external ref
    :param value: float or dict of locations to floats
    :param context: string
    :param location: string, ignored if value is dict
    :param kwargs: overwrite=False, origin=query_quantity.origin, others?
    :return:

